## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-06-17-44-23 +0000,startribune,The making of J.J. McCarthy and the big busine...,https://www.startribune.com/minnesota-vikings-...
1,2025-09-06-17-41-00 +0000,wsj,Larry Ellison Is Spending Billions to Reshape ...,https://www.wsj.com/business/larry-ellison-is-...
2,2025-09-06-17-39-54 +0000,nyt,Hundreds of Pro-Palestinian Protesters Arreste...,https://www.nytimes.com/2025/09/06/world/europ...
3,2025-09-06-17-24-45 +0000,bbc,Mourners pay tribute at public viewing of Gior...,https://www.bbc.com/news/videos/cnvrd46e4q1o?a...
4,2025-09-06-17-18-20 +0000,nypost,Who are the daring Ukrainian ‘Angels’ who save...,https://nypost.com/2025/09/06/world-news/four-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2354/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
93,trump,45
221,new,15
40,immigration,11
98,war,10
403,will,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
176,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...,92
107,2025-09-06-09-00-00 +0000,wsj,Rep. Marjorie Taylor Greene is walking a tight...,https://www.wsj.com/politics/policy/marjorie-t...,80
187,2025-09-05-22-15-00 +0000,wsj,Opinion | Can Trump’s ‘War Department’ Win a War?,https://www.wsj.com/opinion/department-of-war-...,76
204,2025-09-05-21-32-15 +0000,bbc,Trump rebrands Department of Defense as Depart...,https://www.bbc.com/news/articles/cgr9r4qr0ppo...,74
155,2025-09-06-00-36-12 +0000,nyt,Trump Says U.S. Military Has ‘Never Fought to ...,https://www.nytimes.com/2025/09/05/us/politics...,73


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
176,92,2025-09-05-22-57-00 +0000,wsj,President Trump signed an executive order to r...,https://www.wsj.com/politics/national-security...
25,54,2025-09-06-15-27-41 +0000,nyt,South Koreans Are Swept Up in Immigration Raid...,https://www.nytimes.com/2025/09/05/us/georgia-...
216,42,2025-09-05-20-58-03 +0000,nypost,Top secret SEAL Team 6 killed North Korean civ...,https://nypost.com/2025/09/05/us-news/top-secr...
231,40,2025-09-05-20-08-47 +0000,nyt,Elon Musk Could Become First Trillionaire Unde...,https://www.nytimes.com/2025/09/05/business/el...
93,37,2025-09-06-09-46-41 +0000,bbc,'No doubt' Russia will cross Nato border if Uk...,https://www.bbc.com/news/videos/cq65410424qo?a...
140,37,2025-09-06-02-17-09 +0000,nypost,Ex-FBI official tipped off Chinese firm doing ...,https://nypost.com/2025/09/05/us-news/ex-fbi-o...
48,34,2025-09-06-13-28-40 +0000,nypost,Manhattan DA demands sicko NYC super return $1...,https://nypost.com/2025/09/06/us-news/sicko-su...
168,32,2025-09-05-23-19-00 +0000,wsj,The IT unemployment rate fell to 4.5% in Augus...,https://www.wsj.com/articles/it-unemployment-f...
9,28,2025-09-06-17-08-49 +0000,nypost,"Speakers at Detroit Jew-hating conference, inc...",https://nypost.com/2025/09/06/us-news/speakers...
251,28,2025-09-05-18-49-45 +0000,nypost,US-backed Gaza group says boy thought to be ki...,https://nypost.com/2025/09/05/world-news/us-ba...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
